In [1]:
from helper_functions import Helper
from typing import Dict
import pandas as pd
import numpy as np
import pickle
import gc

In [2]:
def formPanel(cw_in_fp: str, ca_panel_in_fp: str, cm_panel_in_fp: str, cmc_panel_in_fp: str, cg_panel_in_fp: str,
    san_panel_in_fp: str, asset_ico_in_fp: str, messari_in_fp: str) -> pd.DataFrame:
    """ Form a single panel DataFrame from all the data sources. """

    # import
    cw_df = pd.read_pickle(cw_in_fp)
    ca_panel_df = pd.read_pickle(ca_panel_in_fp)
    cm_panel_df = pd.read_pickle(cm_panel_in_fp)
    cmc_panel_df = pd.read_pickle(cmc_panel_in_fp)
    cg_panel_df = pd.read_pickle(cg_panel_in_fp)
    san_panel_df = pd.read_pickle(san_panel_in_fp)
    ico_panel_df = pd.read_pickle(asset_ico_in_fp)
    messari_df = pd.read_pickle(messari_in_fp)

    # Merge CM and CA data together
    panel_df = ca_panel_df.merge(cw_df[['asset_ca', 'asset_cm']], on='asset_ca', how='left', validate='many_to_one')
    assert panel_df.shape[0] == ca_panel_df.shape[0]

    nrows_before = panel_df.shape[0]
    panel_df = panel_df.merge(cm_panel_df, on=['date', 'asset_cm'], how='left', validate='one_to_one')
    assert nrows_before <= panel_df.shape[0]

    # Merge on rest of panel data
    nrows_before = panel_df.shape[0]
    san_panel_df = san_panel_df.merge(cw_df[cw_df.asset_san.notnull()][['asset_san', 'asset_cm']], 
                                    on='asset_san', how='inner', validate='many_to_one')
    panel_df = panel_df.merge(san_panel_df, on=['date', 'asset_cm'], how='left', validate='one_to_one')
    cmc_panel_df = cmc_panel_df.merge(cw_df[cw_df.asset_cmc.notnull()][['asset_cmc', 'asset_cm']], 
                                    on='asset_cmc', how='inner', validate='many_to_one')
    panel_df = panel_df.merge(cmc_panel_df, on=['date', 'asset_cm'], how='left', validate='one_to_one')
    cg_panel_df = cg_panel_df.merge(cw_df[cw_df.asset_cg.notnull()][['asset_cg', 'asset_cm']], 
                                    on='asset_cg', how='inner', validate='many_to_one')
    panel_df = panel_df.merge(cg_panel_df, on=['date', 'asset_cm'], how='left', validate='one_to_one')
    panel_df = panel_df.merge(ico_panel_df, on=['asset_cm'], how='left', validate='many_to_one')
    panel_df = panel_df.merge(messari_df, on=['asset_cm'], how='left', validate='many_to_one')

    # Make sure panel wasn't messed up
    assert nrows_before == panel_df.shape[0]

    # Drop other asset id columns
    panel_df = panel_df.drop(['asset_ca', 'asset_san', 'asset_cmc', 'asset_cg'], axis=1)

    # Rename id column
    panel_df = panel_df.rename(columns={'asset_cm': 'asset'})

    # Ensure all columns have char in front of the col name
    panel_df.columns = [col if col in ['date', 'asset'] else 'char_'+col for col in panel_df.columns]

    # Make sure unique on id columns
    assert 0 == panel_df[['date', 'asset']].duplicated().sum()

    # Resort
    cols = list(panel_df.columns.values)
    cols.remove('date')
    cols.remove('asset')
    panel_df = panel_df[['date', 'asset']+cols]
    panel_df = panel_df.sort_values(by=['date', 'asset'], ignore_index=True)

    # Clear memory
    del cw_df, ca_panel_df, cm_panel_df, cmc_panel_df, 
    del cg_panel_df, san_panel_df, ico_panel_df, messari_df
    gc.collect()

    return panel_df

In [3]:
def formMacro(macro_in_fp: str, ca_macro_in_fp: str, cm_macro_in_fp: str, 
    cmc_macro_in_fp: str, san_macro_in_fp: str, macro_ico_in_fp: str) -> pd.DataFrame:
    """ Form single macro DataFrame out of all macro data. """

    # import
    macro_df = pd.read_pickle(macro_in_fp)
    ca_macro_df = pd.read_pickle(ca_macro_in_fp)
    cm_macro_df = pd.read_pickle(cm_macro_in_fp)
    cmc_macro_df = pd.read_pickle(cmc_macro_in_fp)
    san_macro_df = pd.read_pickle(san_macro_in_fp)
    ico_macro_df = pd.read_pickle(macro_ico_in_fp)

    # convert macro to hourly dataframe
    assert 0 == macro_df.isnull().sum().sum()
    macro_df.set_index('date', inplace=True)
    min_dt, max_dt = macro_df.index.min(), macro_df.index.max()
    full_date_range = pd.date_range(start=min_dt, end=max_dt, freq='1H')
    macro_df = macro_df.reindex(full_date_range)
    macro_df = macro_df.ffill()
    macro_df = macro_df.reset_index()
    macro_df = macro_df.rename(columns={'index': 'date'})
    assert 0 == macro_df.isnull().sum().sum()

    # merge on other macro data
    macro_df = macro_df.merge(ca_macro_df, on='date', how='left', validate='one_to_one')
    macro_df = macro_df.merge(cm_macro_df, on='date', how='left', validate='one_to_one')
    macro_df = macro_df.merge(cmc_macro_df, on='date', how='left', validate='one_to_one')
    macro_df = macro_df.merge(san_macro_df, on='date', how='left', validate='one_to_one')
    macro_df = macro_df.merge(ico_macro_df, on='date', how='left', validate='one_to_one')

    # Ensure all columns have char in front of the col name
    macro_df.columns = [col if col=='date' else 'macro_'+col for col in macro_df.columns]

    # Make sure unique on id columns
    assert macro_df.date.is_unique

    # Resort
    macro_df = macro_df.sort_values(by='date', ignore_index=True)

    # Clear memory
    del ca_macro_df, cm_macro_df, cmc_macro_df, san_macro_df, ico_macro_df
    gc.collect()

    return macro_df

In [4]:
def finalizeAssetUniverse(panel_df: pd.DataFrame, asset_universe_dict: Dict[str, list]) -> tuple:
    """ Finalize the asset universe by confirming if every asset and start_of_month_date
        have the necessary 90 days of data before that month; subset the panel to these
        included asset-dates 90 days before and the month of that asset-start_of_month_date.

    Note: this takes about twenty-five minutes to run.

    Args:
        panel_df (pd.DataFrame): the study's panel data.
        asset_universe_dict (dict): dictionary with keys for the start of each month of
            the study period and values of included assets for that month.

    Returns: tuple
        panel_df (pd.DataFrame): subsetted panel to the applicable asset-timestamps.
        final_asset_universe_dict (dict): updated included assets and start_of_month_dates.
    """
    # Initialize dataframe for final date-asset pairs and a new asset_universe_dict
    dates_df = pd.DataFrame(columns=["date", "asset"])
    final_asset_universe_dict = {}
    for date in asset_universe_dict.keys():
        # add this date to the new dictionary
        final_asset_universe_dict[date] = []

        # loop over all assets for this start of month date
        for asset in asset_universe_dict[date]:
            # form start and end date for window 90 days before trading date
            start_date = np.datetime64(date) - np.timedelta64(90, 'D')
            end_date = np.datetime64(date)
            end_date_plus_one_month = end_date + pd.DateOffset(months=1)

            # Form asset dataframe for the dates associated with this asset trading window,
            # including the entire month for this month of the study.
            asset_df = panel_df.loc[(panel_df["asset"] == asset) 
                & (panel_df["date"] >= start_date) 
                & (panel_df["date"] < end_date_plus_one_month)][['date', 'asset']]

            # Count number of obs for just the trailing 3 months
            asset_num_obs = asset_df[(asset_df["date"] >= start_date) 
                                    & (asset_df["date"] < end_date)].shape[0]

            # If asset has all trailing 90 days of data (at hourly freq)
            if asset_num_obs == 2160:
                # then add it to the final asset universe
                final_asset_universe_dict[date].append(asset)

                # and build asset-timestamps to subset the panel down to
                dates_df = pd.concat([dates_df, asset_df])

        # cut down size of dates
        dates_df.drop_duplicates(inplace=True)
    
    # cut panel down to just asset-dates of interest
    panel_df = panel_df.merge(dates_df, on=['date', 'asset'], how='inner', validate='one_to_one')

    return panel_df, final_asset_universe_dict

In [5]:
def combinePanelAndMacro(panel_df: pd.DataFrame, macro_df: pd.DataFrame) -> pd.DataFrame:
    """ Perform various checks and combine the panel and macro data. """
    # merge on macro variables
    panel_df = panel_df.merge(macro_df, on='date', how='left', validate='many_to_one')

    # ensure i have all hours
    min_dt, max_dt = panel_df.date.min(), panel_df.date.max()
    full_date_range = pd.date_range(start=min_dt, end=max_dt, freq='1H')
    assert len(panel_df.date.unique()) == len(full_date_range)

    # ensure date column is clean
    assert 0 == panel_df.date.isnull().sum()
    assert len(panel_df) == panel_df[panel_df.date.dt.minute==0].shape[0]

    # ensure asset column is clean
    asset_universe = Helper.findUniqueAssets(asset_universe_dict)
    assert 0 == panel_df.asset.isnull().sum()
    assert len(panel_df) == panel_df[panel_df.asset.isin(asset_universe)].shape[0]

    # ensure no duplicates
    assert not panel_df.duplicated(subset=['date', 'asset']).any()

    # sort by date then asset and reset index
    panel_df = panel_df.sort_values(by=['date', 'asset'], ignore_index=True)

    # sort columns
    cols = list(panel_df.columns.values)
    cols_to_remove = ['date', 'asset', 
        'char_ico_date_momtaz', 'char_ico_price_momtaz', 'char_ico_momtaz',
        'char_category_san', 'char_industry_messari', 
        'char_pow_messari', 'char_pos_messari',
        'char_asset_usage_payments_messari', 'char_asset_usage_vote_messari',
        'char_asset_usage_work_messari', 'char_asset_usage_dividends_messari',
        'char_asset_usage_access_messari', 'char_asset_usage_discount_messari']
    for col in cols_to_remove:
        cols.remove(col)
    panel_df = panel_df[cols_to_remove+cols]

    # ensure no missing in critical columns
    critical_cols = ['char_usd_per_token_ca',
        'char_usd_volume_ca',
        'char_usd_per_token_cm',
        'char_usd_volume_cm']
    for col in critical_cols:
        assert 1000 > panel_df[col].isnull().sum()

    panel_df.loc[panel_df.char_usd_per_token_cm.isnull(), 'char_usd_per_token_cm'] = panel_df.char_usd_per_token_ca
    panel_df.loc[panel_df.char_usd_volume_cm.isnull(), 'char_usd_volume_cm'] = panel_df.char_usd_volume_ca

    critical_cols = ['char_usd_per_token_ca',
        'char_usd_volume_ca',
        'char_usd_per_token_cm',
        'char_usd_volume_cm']
    for col in critical_cols:
        assert 0 == panel_df[col].isnull().sum()

    return panel_df

In [6]:
def formPrices(panel_df: pd.DataFrame) -> pd.DataFrame:
    """ Form price column from actually tradable prices and 
        the global price from all data sources. 
    
    Args: 
        panel_df (pd.DataFrame): the panel data frame with all columns.
        
    Returns:
        (pd.DataFrame): updated panel data with new columns char_price_t
                        and char_price_global_t with the raw prices 
                        columns removed.
    """
    # confirm cm and ca tradable prices are good to go
    assert 0 == panel_df.char_usd_per_token_ca.isnull().sum()
    assert 0 == panel_df.char_usd_per_token_cm.isnull().sum()
    assert 0 == (panel_df.char_usd_per_token_ca==0).sum()
    panel_df.loc[panel_df.char_usd_per_token_cm==0, 'char_usd_per_token_cm'] = np.nan
    assert 1e6 > np.max(panel_df.char_usd_per_token_cm)
    assert 1e6 > np.max(panel_df.char_usd_per_token_ca)
    assert 0 < np.min(panel_df.char_usd_per_token_cm)
    assert 0 < np.min(panel_df.char_usd_per_token_ca)

    # form char_price_t
    panel_df['char_price_t'] = np.nan
    panel_df.loc[panel_df.char_usd_per_token_cm.isnull(), 'char_price_t'] = panel_df.char_usd_per_token_ca
    panel_df.loc[panel_df.char_usd_per_token_cm.notnull(), 'char_price_t'] = (1/2)*(panel_df.char_usd_per_token_cm
                                                                            + panel_df.char_usd_per_token_ca)
    assert 0 == panel_df.char_price_t.isnull().sum()
    assert 0 < np.min(panel_df.char_price_t)
    assert 1e6 > np.min(panel_df.char_price_t)

    # clean up old cm and ca columns
    panel_df = panel_df.drop(['char_usd_per_token_ca', 'char_usd_per_token_cm'], axis=1)

    # clean up the global price cols before combining
    global_price_cols = ['char_usd_ref_price_ca', 'char_reference_rate_usd_cm',
        'char_price_usd_san', 'char_usd_per_token_cmc', 'char_usd_per_token_cg']
    for col in global_price_cols:
        panel_df.loc[(panel_df[col]<=0) | (panel_df[col]>1e6) , col] = np.nan

    # form global price column
    panel_df['char_price_global_t'] = panel_df[global_price_cols].mean(axis=1)
    assert 0 == panel_df.char_price_global_t.isnull().sum()
    assert 0 == panel_df[(panel_df.char_price_global_t <= 0) | (panel_df.char_price_global_t > 1e6)].shape[0]

    # clean up old global price columns
    panel_df = panel_df.drop(global_price_cols, axis=1)

    return panel_df


In [16]:
def formLHSs(panel_df: pd.DataFrame) -> pd.DataFrame:
    """ Form LHS's of excess returns one hour and day ahead. """

    # Form one hour ahead risk free return
    panel_df['r_rf_tp1'] = (1+panel_df.macro_dgs1mo_fred.values/100)**(1/(365*24))-1

    # Form one day ahead risk free return
    panel_df['r_rf_tp24'] = (1+panel_df.macro_dgs1mo_fred.values/100)**(1/(365))-1

    # Form one hour ahead excess return

    # form new df of the prices for one hour ahead
    temp_df = panel_df[['date', 'asset', 'char_price_t']].copy()
    temp_df = temp_df.rename(columns={'char_price_t': 'char_price_tp1'})
    temp_df['date'] = temp_df['date'] - pd.to_timedelta(1, unit='H')

    # merge it back on
    panel_df = panel_df.merge(temp_df, on=['date', 'asset'], how='left', validate='one_to_one')
    del temp_df

    # form return
    panel_df['r_ex_tp1'] = (((panel_df.char_price_tp1 
                            - panel_df.char_price_t)/panel_df.char_price_t)
                            - panel_df.r_rf_tp1)

    # Form one day ahead excess return

    # form new df of the prices for one day ahead
    temp_df = panel_df[['date', 'asset', 'char_price_t']].copy()
    temp_df = temp_df.rename(columns={'char_price_t': 'char_price_tp24'})
    temp_df['date'] = temp_df['date'] - pd.to_timedelta(24, unit='H')

    # merge it back on
    panel_df = panel_df.merge(temp_df, on=['date', 'asset'], how='left', validate='one_to_one')
    del temp_df

    # form return
    panel_df['r_ex_tp24'] = (((panel_df.char_price_tp24
                                - panel_df.char_price_t)/panel_df.char_price_t)
                                - panel_df.r_rf_tp24)

    # Drop unnecssary columns
    panel_df = panel_df.drop(columns={'r_rf_tp1', 'r_rf_tp24', 'char_price_tp1', 'char_price_tp24'})

    # Drop rows with missing lhs
    panel_df = panel_df[panel_df.r_ex_tp24.notnull()]
    panel_df = panel_df[panel_df.r_ex_tp1.notnull()]

    return panel_df


In [7]:
if __name__ == "__main__":
    # set args
    CW_IN_FP = '../data/clean/cw.pkl'
    ASSET_IN_FP = '../data/derived/asset_universe_dict.pickle'
    ASSET_OUT_FP = '../data/clean/asset_universe_dict.pickle'
    CA_PANEL_IN_FP = '../data/derived/ca_panel.pkl'
    CA_MACRO_IN_FP = '../data/derived/ca_macro.pkl'
    CM_PANEL_IN_FP = "../data/derived/cm_panel.pkl"
    CM_MACRO_IN_FP = '../data/derived/cm_macro.pkl'
    CG_PANEL_IN_FP = '../data/derived/cg_panel.pkl'
    CMC_PANEL_IN_FP = '../data/derived/cmc_panel.pkl'
    CMC_MACRO_IN_FP = '../data/derived/cmc_macro.pkl'
    SAN_PANEL_IN_FP = "../data/derived/san_panel.pkl"
    SAN_MACRO_IN_FP = '../data/derived/san_macro.pkl'
    MACRO_IN_FP     = '../data/derived/macro.pkl'
    ASSET_ICO_IN_FP = '../data/derived/momtaz_ico_asset.pkl' 
    MACRO_ICO_IN_FP = '../data/derived/momtaz_ico_macro.pkl' 
    MESSARI_IN_FP = '../data/derived/messari.pkl'

    # import
    with open(ASSET_IN_FP, "rb") as f:
        asset_universe_dict = pickle.load(f)
    
    # form panel
    panel_df = formPanel(CW_IN_FP, CA_PANEL_IN_FP, CM_PANEL_IN_FP, CMC_PANEL_IN_FP,
                CG_PANEL_IN_FP, SAN_PANEL_IN_FP, ASSET_ICO_IN_FP, MESSARI_IN_FP)

    # form macro
    macro_df = formMacro(MACRO_IN_FP, CA_MACRO_IN_FP, CM_MACRO_IN_FP,
        CMC_MACRO_IN_FP, SAN_MACRO_IN_FP, MACRO_ICO_IN_FP)
    
    # update asset universe
    panel_df, asset_universe_dict = finalizeAssetUniverse(panel_df, asset_universe_dict)
    with open(ASSET_OUT_FP, 'wb') as f:
        pickle.dump(asset_universe_dict, f)

    # combine the panel and macro data
    panel_df = combinePanelAndMacro(panel_df, macro_df)

    # form new columns
    panel_df = formPrices(panel_df)
    panel_df = formLHSs(panel_df)

In [14]:
# TODO MOVE THIS UP FOR THE ROW DROP IF THE ABOVE FOUR CELLS LOOK GOOD


In [15]:
panel_df.shape

(4548925, 588)

In [ ]:
panel_df.r_ex_tp1.describe()

In [ ]:
panel_df.r_ex_tp24.describe()

In [ ]:
panel_df[panel_df.r_ex_tp1>3]

In [ ]:
panel_df[panel_df.r_ex_tp24>3]

In [ ]:
panel_df[panel_df.r_ex_tp1<-.9]

In [ ]:
panel_df[panel_df.r_ex_tp24<-.9]

In [ ]:
# TODO LOOK FOR SUSPECT RETURNS TO POTENTIALLY SET TO ZERO AND FFILL

In [ ]:
# TODO CLEAN VOLUME COLUMNS


 'char_usd_volume_ca',
 'char_trades_volume_ca',

 'char_trades_cm',
 'char_usd_volume_cm',

 'char_usd_volume_24h_cmc',

 'char_usd_volume_24h_cg',

 'char_volume_usd_san',

# TODO COPY THE PRICE CODE TO RE-USE

# Form char_volume_t

# Form char_volume_global_t

# Ensure no missing in new cols

# Drop old volume columns

# do same for trade volume columns

# -confirm column ranges
# -confirm missingness
# -would it be better to have the raw missing from one of the data sources that i am using?
# -would it be helpful to be able to cross check a small number of assets for critical varaibles
#  like supply, mcap, addresses, transfers then if so, pull in raw CM panel
# -consider combining with others?
# -consider colsolidating down? or mayb wait until feat eng time?

In [ ]:
# TODO CLEAN MCAP COLUMNS


 'char_cap_mrkt_est_usd_cm',
 'char_usd_mcap_cmc',
 'char_usd_mcap_cg',


 'char_marketcap_usd_san',

# TODO COPY THE VOLUME CODE TO RE-USE

# Form char_mcap_t

# Ensure no missing

# Drop old mcap columns

# TODO MAYBE KEEP MANY OF THEM ACTUALLY?

# -confirm column ranges
# -confirm missingness
# -would it be better to have the raw missing from one of the data sources that i am using?
# -would it be helpful to be able to cross check a small number of assets for critical varaibles
#  like supply, mcap, addresses, transfers then if so, pull in raw CM panel
# -consider combining with others?
# -consider colsolidating down? or mayb wait until feat eng time?

In [10]:
# TODO FORM SINGLE INDUSTRY COLUMN AND SINGLE USAGE COL
# -category_san
# TODO CLEAN MESSARI COLUMNS
# TODO CLEAN MOMTAZ COLUMNS


 'char_ico_date_momtaz',
 'char_ico_price_momtaz',
 'char_ico_momtaz',
 
 'char_category_san',
 'char_industry_messari',
 
 'char_pow_messari',
 'char_pos_messari',
 
 'char_asset_usage_payments_messari',
 'char_asset_usage_vote_messari',
 'char_asset_usage_work_messari',
 'char_asset_usage_dividends_messari',
 'char_asset_usage_access_messari',
 'char_asset_usage_discount_messari',

# TODO CLEAN REST OF STATIC CHAR COLS

# -put all of these columns after date, asset, LHS's, price, volume, mcap



# -confirm column ranges
# -confirm missingness
# -would it be better to have the raw missing from one of the data sources that i am using?
# -would it be helpful to be able to cross check a small number of assets for critical varaibles
#  like supply, mcap, addresses, transfers then if so, pull in raw CM panel
# -consider combining with others?
# -consider colsolidating down? or mayb wait until feat eng time?


In [ ]:
# TODO CLEAN SUPPLY COLUMNS


 'char_circulating_supply_cmc',
 'char_max_supply_cmc',
 'char_total_supply_cmc',

 'char_total_supply_san',

In [ ]:
# TODO CLEAN GITHUB VAR


 'char_github_activity_cg',
 'char_github_activity_san',
 'char_github_activity_contributors_count_san',

'char_dev_activity_san',
 'char_dev_activity_contributors_count_san',

In [ ]:
# TODO CLEAN SOCIAL VARS


 'char_reddit_activity_cg',
 'char_twitter_followers_cg',

 'char_unique_social_volume_total_1h_san',

  'char_sentiment_balance_reddit_san',
 'char_sentiment_balance_total_san',
 'char_sentiment_balance_twitter_san',
 'char_sentiment_balance_twitter_crypto_san',

 'char_sentiment_negative_reddit_san',
 'char_sentiment_negative_total_san',
 'char_sentiment_negative_twitter_san',
 'char_sentiment_negative_twitter_crypto_san',
 'char_sentiment_positive_reddit_san',
 'char_sentiment_positive_total_san',
 'char_sentiment_positive_twitter_san',
 'char_sentiment_positive_twitter_crypto_san',

 'char_sentiment_volume_consumed_reddit_san',
 'char_sentiment_volume_consumed_total_san',
 'char_sentiment_volume_consumed_twitter_san',
 'char_sentiment_volume_consumed_twitter_crypto_san',

 'char_social_dominance_reddit_san',
 'char_social_dominance_total_san',
 'char_social_dominance_twitter_san',
 'char_social_dominance_twitter_crypto_san',

 'char_social_volume_reddit_san',
 'char_social_volume_total_san',
 'char_social_volume_twitter_san',
 'char_social_volume_twitter_crypto_san',

In [ ]:
# TODO CLEAN CMC COLUMNS
# -confirm column ranges
# -confirm missingness
# -would it be better to have the raw missing from one of the data sources that i am using?
# -would it be helpful to be able to cross check a small number of assets for critical varaibles
#  like supply, mcap, addresses, transfers then if so, pull in raw CM panel
# -consider combining with others?
# -consider colsolidating down? or mayb wait until feat eng time?



 'char_num_market_pairs_cmc',
 'char_rank_cmc',
 'char_vc_cmc',

In [ ]:
# TODO CLEAN SAN COLUMNS
# -confirm column ranges
# -confirm missingness
# -would it be better to have the raw missing from one of the data sources that i am using?
# -would it be helpful to be able to cross check a small number of assets for critical varaibles
#  like supply, mcap, addresses, transfers then if so, pull in raw CM panel
# -consider combining with others?
# -consider colsolidating down? or mayb wait until feat eng time?




In [ ]:
# TODO SAN

 'char_daily_active_addresses_san',
 'char_active_addresses_1h_san',

  'char_network_growth_san',
 
 'char_payments_count_san',

 'char_transaction_volume_san',
 'char_transactions_count_san',


In [ ]:
# TODO SAN




 'char_active_deposits_san',
 'char_active_deposits_per_exchange_san',
 'char_active_withdrawals_san',
 'char_active_withdrawals_per_exchange_san',

  'char_withdrawal_transactions_san',

 'char_deposit_balance_san',
 'char_deposit_transactions_san',
 'char_deposit_transactions_per_exchange_san',



In [ ]:

# TODO SAN


 'char_age_consumed_san',
 'char_age_destroyed_san',

 'char_mean_age_san',
 'char_mean_dollar_invested_age_san',



In [ ]:
# TODO SAN

 'char_cexes_to_defi_flow_san',
 'char_cexes_to_dex_flow_san',
 'char_cexes_to_dex_traders_flow_san',
 'char_cexes_to_traders_flow_san',
 'char_cexes_to_whale_flow_san',

 'char_defi_to_cexes_flow_san',
 'char_defi_to_dex_traders_flow_san',
 'char_defi_to_dexes_flow_san',
 'char_defi_to_exchanges_flow_san',
 'char_defi_to_traders_flow_san',
 'char_defi_to_whale_flow_san',

 'char_dex_to_cexes_flow_san',

 'char_dex_traders_to_cexes_flow_san',
 'char_dex_traders_to_defi_flow_san',
 'char_dex_traders_to_dexes_flow_san',
 'char_dex_traders_to_exchanges_flow_san',
 'char_dex_traders_to_whale_flow_san',

 'char_dexes_to_defi_flow_san',
 'char_dexes_to_dex_traders_flow_san',
 'char_dexes_to_traders_flow_san',
 'char_dexes_to_whale_flow_san',

 'char_exchange_inflow_san',
 'char_exchange_inflow_usd_san',
 'char_exchange_outflow_san',
 'char_exchange_outflow_usd_san',

 'char_exchanges_to_defi_flow_san',
 'char_exchanges_to_dex_traders_flow_san',
 'char_exchanges_to_genesis_flow_san',
 'char_exchanges_to_traders_flow_san',
 'char_exchanges_to_whales_flow_san',

 'char_traders_to_cexes_flow_san',
 'char_traders_to_defi_flow_san',
 'char_traders_to_dexes_flow_san',
 'char_traders_to_exchanges_flow_san',
 'char_traders_to_whale_flow_san',

 'char_whale_to_cexes_flow_san',
 'char_whale_to_defi_flow_san',
 'char_whale_to_dex_traders_flow_san',
 'char_whale_to_dexes_flow_san',
 'char_whale_to_traders_flow_san',
 'char_whales_to_exchanges_flow_san',

In [ ]:
# TODO SAN

 'char_holders_distribution_combined_balance_over_1_san',
 'char_holders_distribution_combined_balance_over_10_san',
 'char_holders_distribution_combined_balance_over_100_san',
 'char_holders_distribution_combined_balance_over_100k_san',
 'char_holders_distribution_combined_balance_over_10k_san',
 'char_holders_distribution_combined_balance_over_1M_san',
 'char_holders_distribution_combined_balance_over_1k_san',
 'char_holders_distribution_combined_balance_total_san',
 'char_holders_distribution_over_1_san',
 'char_holders_distribution_over_10_san',
 'char_holders_distribution_over_100_san',
 'char_holders_distribution_over_100k_san',
 'char_holders_distribution_over_10k_san',
 'char_holders_distribution_over_1M_san',
 'char_holders_distribution_over_1k_san',
 'char_holders_distribution_total_san',

 'char_active_holders_distribution_combined_balance_over_1_san',
 'char_active_holders_distribution_combined_balance_over_10_san',
 'char_active_holders_distribution_combined_balance_over_100_san',
 'char_active_holders_distribution_combined_balance_over_100k_san',
 'char_active_holders_distribution_combined_balance_over_10k_san',
 'char_active_holders_distribution_combined_balance_over_1M_san',
 'char_active_holders_distribution_combined_balance_over_1k_san',
 'char_active_holders_distribution_combined_balance_total_san',
 'char_active_holders_distribution_over_1_san',
 'char_active_holders_distribution_over_10_san',
 'char_active_holders_distribution_over_100_san',
 'char_active_holders_distribution_over_100k_san',
 'char_active_holders_distribution_over_10k_san',
 'char_active_holders_distribution_over_1M_san',
 'char_active_holders_distribution_over_1k_san',
 'char_active_holders_distribution_total_san',



In [ ]:
# TODO SAN
 
 'char_all_known_balance_san',
 'char_amount_in_exchange_top_holders_san',
 'char_amount_in_non_exchange_top_holders_san',
 'char_amount_in_top_holders_san',
 'char_cex_balance_san',
 
 'char_defi_balance_san',
 'char_defi_cex_balance_san',
 'char_defi_dex_balance_san',
 'char_defi_exchange_balance_san',

 'char_dex_balance_san',
 'char_dex_cex_balance_san',

 'char_dex_trader_balance_san',
 'char_dex_traders_cex_balance_san',
 'char_dex_traders_defi_balance_san',
 'char_dex_traders_dex_balance_san',
 'char_dex_traders_exchange_balance_san',

 'char_dex_traders_whale_balance_san',

 'char_exchange_balance_san',

  'char_percent_of_total_supply_on_exchanges_san',

 'char_supply_on_exchanges_san',
 'char_supply_outside_exchanges_san',

 'char_trader_balance_san',
 'char_traders_cex_balance_san',
 'char_traders_defi_balance_san',
 'char_traders_dex_balance_san',
 'char_traders_exchange_balance_san',

 'char_traders_whale_balance_san',

 'char_whale_balance_san',
 'char_whale_cex_balance_san',
 'char_whale_defi_balance_san',
 'char_whale_dex_balance_san',

 'char_whales_exchange_balance_san',

 'char_withdrawal_balance_san',


In [ ]:
# TODO SAN

 'char_circulation_san',
 'char_circulation_1d_san',
 'char_circulation_2y_san',
 'char_circulation_30d_san',
 'char_circulation_365d_san',
 'char_circulation_3y_san',
 'char_circulation_5y_san',
 'char_circulation_7d_san',
 'char_circulation_90d_san',

 'char_dormant_circulation_180d_san',
 'char_dormant_circulation_365d_san',
 'char_dormant_circulation_90d_san',
 





In [ ]:
# TODO SAN


 
 
 'char_stock_to_flow_san',

 'char_mean_realized_price_usd_san',
 'char_mvrv_long_short_diff_usd_san',
 'char_mvrv_usd_san',
 'char_nvt_san',
 'char_nvt_transaction_volume_san',

 'char_realized_value_usd_san',
 

 'char_percent_of_total_supply_in_profit_san',

 'char_total_supply_in_profit_san',

In [ ]:
# TODO CLEAN CA USDC AND USDT INTO DEV FROM 1 DOLLAR AND COMBINE WITH OTHERS
# -confirm column ranges
# -confirm missingness
# -would it be better to have the raw missing from one of the data sources that i am using?
# -would it be helpful to be able to cross check a small number of assets for critical varaibles
#  like supply, mcap, addresses, transfers then if so, pull in raw CM panel
# -consider combining with others?
# -consider colsolidating down? or mayb wait until feat eng time?


 'macro_usdt_usdc_dev_from_one_cm',

 
 'macro_usd_per_usdc_ca',
 'macro_usd_per_usdt_ca',


In [ ]:
# TODO CLEAN MACRO MOMTAZ COLUMNS

# -confirm column ranges
# -confirm missingness
# -would it be better to have the raw missing from one of the data sources that i am using?
# -would it be helpful to be able to cross check a small number of assets for critical varaibles
#  like supply, mcap, addresses, transfers then if so, pull in raw CM panel
# -consider combining with others?
# -consider colsolidating down? or mayb wait until feat eng time?


 'macro_ico_sum_momtaz'

In [7]:
# TODO CLEAN MACRO FED COLUMNS
# -confirm column ranges
# -confirm missingness
# -would it be better to have the raw missing from one of the data sources that i am using?
# -would it be helpful to be able to cross check a small number of assets for critical varaibles
#  like supply, mcap, addresses, transfers then if so, pull in raw CM panel
# -consider combining with others?
# -consider colsolidating down? or mayb wait until feat eng time?



 'macro_aaa_fed',
 'macro_aaaffm_fed',
 'macro_acogno_fed',
 'macro_amdmnox_fed',
 'macro_amdmuox_fed',
 'macro_andenox_fed',
 'macro_awhman_fed',
 'macro_awotman_fed',
 'macro_baa_fed',
 'macro_baaffm_fed',
 'macro_bogmbase_fed',
 'macro_businvx_fed',
 'macro_busloans_fed',
 'macro_ce16ov_fed',
 'macro_ces0600000007_fed',
 'macro_ces0600000008_fed',
 'macro_ces1021000001_fed',
 'macro_ces2000000008_fed',
 'macro_ces3000000008_fed',
 'macro_claimsx_fed',
 'macro_clf16ov_fed',
 'macro_cmrmtsplx_fed',
 'macro_compapffx_fed',
 'macro_conspi_fed',
 'macro_cp3mx_fed',
 'macro_cpiappsl_fed',
 'macro_cpiaucsl_fed',
 'macro_cpimedsl_fed',
 'macro_cpitrnsl_fed',
 'macro_cpiulfsl_fed',
 'macro_cumfns_fed',
 'macro_cusr0000sa0l2_fed',
 'macro_cusr0000sa0l5_fed',
 'macro_cusr0000sac_fed',
 'macro_cusr0000sad_fed',
 'macro_cusr0000sas_fed',
 'macro_ddurrg3m086sbea_fed',
 'macro_dmanemp_fed',
 'macro_dndgrg3m086sbea_fed',
 'macro_dpcera3m086sbea_fed',
 'macro_dserrg3m086sbea_fed',
 'macro_dtcolnvhfnm_fed',
 'macro_dtcthfnm_fed',
 'macro_excausx_fed',
 'macro_exjpusx_fed',
 'macro_exszusx_fed',
 'macro_exusukx_fed',
 'macro_fedfunds_fed',
 'macro_gs1_fed',
 'macro_gs10_fed',
 'macro_gs5_fed',
 'macro_houst_fed',
 'macro_houstmw_fed',
 'macro_houstne_fed',
 'macro_housts_fed',
 'macro_houstw_fed',
 'macro_hwi_fed',
 'macro_hwiuratio_fed',
 'macro_indpro_fed',
 'macro_invest_fed',
 'macro_ipb51222s_fed',
 'macro_ipbuseq_fed',
 'macro_ipcongd_fed',
 'macro_ipdcongd_fed',
 'macro_ipdmat_fed',
 'macro_ipfinal_fed',
 'macro_ipfpnss_fed',
 'macro_ipfuels_fed',
 'macro_ipmansics_fed',
 'macro_ipmat_fed',
 'macro_ipncongd_fed',
 'macro_ipnmat_fed',
 'macro_isratiox_fed',
 'macro_m1sl_fed',
 'macro_m2real_fed',
 'macro_m2sl_fed',
 'macro_manemp_fed',
 'macro_ndmanemp_fed',
 'macro_nonborres_fed',
 'macro_nonrevsl_fed',
 'macro_oilpricex_fed',
 'macro_payems_fed',
 'macro_pcepi_fed',
 'macro_permit_fed',
 'macro_permitmw_fed',
 'macro_permitne_fed',
 'macro_permits_fed',
 'macro_permitw_fed',
 'macro_ppicmm_fed',
 'macro_realln_fed',
 'macro_retailx_fed',
 'macro_rpi_fed',
 'macro_s&p 500_fed',
 'macro_s&p pe ratio_fed',
 'macro_s&p div yield_fed',
 'macro_s&p: indust_fed',
 'macro_srvprd_fed',
 'macro_t10yffm_fed',
 'macro_t1yffm_fed',
 'macro_t5yffm_fed',
 'macro_tb3ms_fed',
 'macro_tb3smffm_fed',
 'macro_tb6ms_fed',
 'macro_tb6smffm_fed',
 'macro_totresns_fed',
 'macro_twexafegsmthx_fed',
 'macro_uemp15ov_fed',
 'macro_uemp15t26_fed',
 'macro_uemp27ov_fed',
 'macro_uemp5to14_fed',
 'macro_uemplt5_fed',
 'macro_uempmean_fed',
 'macro_umcsentx_fed',
 'macro_unrate_fed',
 'macro_uscons_fed',
 'macro_usfire_fed',
 'macro_usgood_fed',
 'macro_usgovt_fed',
 'macro_ustpu_fed',
 'macro_ustrade_fed',
 'macro_uswtrade_fed',
 'macro_vixclsx_fed',
 'macro_w875rx1_fed',
 'macro_wpsfd49207_fed',
 'macro_wpsfd49502_fed',
 'macro_wpsid61_fed',
 'macro_wpsid62_fed',

In [ ]:
# TODO CLEAN MACRO FRED COLUMNS
# -confirm column ranges
# -confirm missingness
# -would it be better to have the raw missing from one of the data sources that i am using?
# -would it be helpful to be able to cross check a small number of assets for critical varaibles
#  like supply, mcap, addresses, transfers then if so, pull in raw CM panel
# -consider combining with others?
# -consider colsolidating down? or mayb wait until feat eng time?



 'macro_dfii10_fred',
 'macro_dfii20_fred',
 'macro_dfii30_fred',
 'macro_dfii5_fred',
 'macro_dfii7_fred',
 'macro_dgs1mo_fred',
 'macro_expinf10yr_fred',
 'macro_expinf1yr_fred',
 'macro_expinf20yr_fred',
 'macro_expinf2yr_fred',
 'macro_expinf30yr_fred',
 'macro_expinf3yr_fred',
 'macro_expinf5yr_fred',
 'macro_t10yie_fred',
 'macro_t20yiem_fred',
 'macro_t30yiem_fred',
 'macro_t5yie_fred',
 'macro_teu-sca_fred',
 'macro_tmu-sca_fred',
 'macro_emv_fred',
 'macro_emv_inflation_fred',
 'macro_gepu_fred',
 'macro_us_mpu_fred',


In [ ]:
# TODO FIGURE OUT HOW TO COMBINE, IF AT ALL, THE CM, CMC, AND SAN COLS TO COME

In [8]:
# TODO CLEAN MACRO CM COLUMNS

# -confirm column ranges
# -confirm missingness
# -would it be better to have the raw missing from one of the data sources that i am using?
# -would it be helpful to be able to cross check a small number of assets for critical varaibles
#  like supply, mcap, addresses, transfers then if so, pull in raw CM panel
# -consider combining with others?
# -consider colsolidating down? or mayb wait until feat eng time?



 'macro_avg_fee_mean_usd_cm',
 'macro_avg_fee_med_usd_cm',
 'macro_avg_fee_rev_pct_cm',
 'macro_avg_ndf_cm',
 'macro_avg_nvt_adj_ff_cm',
 'macro_avg_rvt_adj_cm',
 'macro_avg_ser_cm',
 'macro_avg_sopr_cm',
 'macro_avg_sply_act_pct_1yr_cm',
 'macro_avg_vel_act_1yr_cm',
 'macro_avg_vel_cur_1yr_cm',

 'macro_btc_adr_act_cnt_cm',
 'macro_btc_adr_act_rec_cnt_cm',
 'macro_btc_adr_act_sent_cnt_cm',
 'macro_btc_adr_bal_cnt_cm',
 'macro_btc_cap_act_1yr_usd_cm',
 'macro_btc_cap_mrkt_ffusd_cm',
 'macro_btc_cap_mvrv_cur_cm',
 'macro_btc_cap_real_usd_cm',
 'macro_btc_diff_mean_cm',
 'macro_btc_fee_med_usd_cm',
 'macro_btc_fee_rev_pct_cm',
 'macro_btc_fee_tot_usd_cm',
 'macro_btc_flow_in_ex_usd_cm',
 'macro_btc_flow_miner_net_0hop_all_usd_cm',
 'macro_btc_flow_out_ex_usd_cm',
 'macro_btc_hash_rate_cm',
 'macro_btc_iss_tot_usd_cm',
 'macro_btc_mcrc_cm',
 'macro_btc_mctc_cm',
 'macro_btc_momr_cm',
 'macro_btc_mri_0hop_all30d_cm',
 'macro_btc_ndf_cm',
 'macro_btc_nvt_adj_ff_cm',
 'macro_btc_puell_mul_rev_cm',
 'macro_btc_puell_mul_tot_cm',
 'macro_btc_rev_hash_usd_cm',
 'macro_btc_rev_usd_cm',
 'macro_btc_rvt_adj_cm',
 'macro_btc_ser_cm',
 'macro_btc_sopr_cm',
 'macro_btc_sply_act_10yr_cm',
 'macro_btc_sply_act_180d_cm',
 'macro_btc_sply_act_1d_cm',
 'macro_btc_sply_act_1yr_cm',
 'macro_btc_sply_act_30d_cm',
 'macro_btc_sply_act_5yr_cm',
 'macro_btc_sply_act_7d_cm',
 'macro_btc_sply_act_ever_cm',
 'macro_btc_sply_act_pct_1yr_cm',
 'macro_btc_sply_adr_bal_usd_100_cm',
 'macro_btc_sply_adr_bal_usd_10k_cm',
 'macro_btc_sply_adr_bal_usd_1_cm',
 'macro_btc_sply_adr_bal_usd_1m_cm',
 'macro_btc_sply_adr_top_100_cm',
 'macro_btc_sply_adr_top_1pct_cm',
 'macro_btc_sply_cur_cm',
 'macro_btc_sply_ex_usd_cm',
 'macro_btc_sply_exp_fut_10yr_cm',
 'macro_btc_sply_ff_cm',
 'macro_btc_sply_miner_0hop_all_usd_cm',
 'macro_btc_sply_rvv_180d_cm',
 'macro_btc_sply_rvv_1yr_cm',
 'macro_btc_sply_rvv_30d_cm',
 'macro_btc_sply_rvv_5yr_cm',
 'macro_btc_sply_rvv_7d_cm',
 'macro_btc_sply_utxo_loss_cm',
 'macro_btc_sply_utxo_prof_cm',
 'macro_btc_tx_tfr_cnt_cm',
 'macro_btc_tx_tfr_val_adj_usd_cm',
 'macro_btc_tx_tfr_val_day_dst_cm',
 'macro_btc_tx_tfr_val_med_usd_cm',
 'macro_btc_tx_tfr_val_usd_cm',
 'macro_btc_utxo_age_med_cm',
 'macro_btc_utxo_loss_unreal_usd_cm',
 'macro_btc_utxo_prof_unreal_usd_cm',
 'macro_btc_vel_act_1yr_cm',
 'macro_btc_vel_cur_1yr_cm',

 'macro_eth_adr_act_cnt_cm',
 'macro_eth_adr_act_cont_cnt_cm',
 'macro_eth_adr_act_rec_cnt_cm',
 'macro_eth_adr_act_sent_cnt_cm',
 'macro_eth_adr_bal_cnt_cm',
 'macro_eth_cap_act_1yr_usd_cm',
 'macro_eth_cap_mrkt_ffusd_cm',
 'macro_eth_cap_mvrv_cur_cm',
 'macro_eth_cap_real_usd_cm',
 'macro_eth_cont_erc_20_cnt_cm',
 'macro_eth_fee_med_usd_cm',
 'macro_eth_fee_rev_pct_cm',
 'macro_eth_fee_tot_usd_cm',
 'macro_eth_flow_in_ex_usd_cm',
 'macro_eth_flow_out_ex_usd_cm',
 'macro_eth_gas_used_tx_cm',
 'macro_eth_iss_tot_usd_cm',
 'macro_eth_ndf_cm',
 'macro_eth_nvt_adj_ff_cm',
 'macro_eth_puell_mul_rev_cm',
 'macro_eth_puell_mul_tot_cm',
 'macro_eth_rev_hash_usd_cm',
 'macro_eth_rev_usd_cm',
 'macro_eth_rvt_adj_cm',
 'macro_eth_ser_cm',
 'macro_eth_sply_act_10yr_cm',
 'macro_eth_sply_act_180d_cm',
 'macro_eth_sply_act_1d_cm',
 'macro_eth_sply_act_1yr_cm',
 'macro_eth_sply_act_30d_cm',
 'macro_eth_sply_act_5yr_cm',
 'macro_eth_sply_act_7d_cm',
 'macro_eth_sply_act_ever_cm',
 'macro_eth_sply_act_pct_1yr_cm',
 'macro_eth_sply_adr_bal_usd_100_cm',
 'macro_eth_sply_adr_bal_usd_10k_cm',
 'macro_eth_sply_adr_bal_usd_1_cm',
 'macro_eth_sply_adr_bal_usd_1m_cm',
 'macro_eth_sply_adr_top_100_cm',
 'macro_eth_sply_adr_top_1pct_cm',
 'macro_eth_sply_burnt_usd_cm',
 'macro_eth_sply_cur_cm',
 'macro_eth_sply_ex_usd_cm',
 'macro_eth_sply_exp_fut_10yr_cm',
 'macro_eth_sply_ff_cm',
 'macro_eth_tx_tfr_cnt_cm',
 'macro_eth_tx_tfr_val_adj_usd_cm',
 'macro_eth_tx_tfr_val_med_usd_cm',
 'macro_eth_tx_tfr_val_usd_cm',
 'macro_eth_vel_act_1yr_cm',
 'macro_eth_vel_cur_1yr_cm',

 'macro_ex_open_interest_future_usd_cm',
 'macro_ex_volume_future_usd_cm',
 'macro_ex_volume_spot_usd_cm',

 'macro_total_adr_act_cnt_cm',
 'macro_total_adr_act_rec_cnt_cm',
 'macro_total_adr_act_sent_cnt_cm',
 'macro_total_adr_bal_cnt_cm',
 'macro_total_adr_bal_usd_100_cnt_cm',
 'macro_total_adr_bal_usd_100k_cnt_cm',
 'macro_total_adr_bal_usd_10k_cnt_cm',
 'macro_total_adr_bal_usd_1m_cnt_cm',
 'macro_total_cap_act_1yr_usd_cm',
 'macro_total_cap_fut_exp_10yr_usd_cm',
 'macro_total_cap_mrkt_cur_usd_cm',
 'macro_total_cap_mrkt_ffusd_cm',
 'macro_total_cap_mvrv_cur_cm',
 'macro_total_cap_real_usd_cm',
 'macro_total_fee_tot_usd_cm',
 'macro_total_iss_tot_usd_cm',
 'macro_total_rev_usd_cm',
 'macro_total_sply_act_10yr_cm',
 'macro_total_sply_act_180d_cm',
 'macro_total_sply_act_1d_cm',
 'macro_total_sply_act_1yr_cm',
 'macro_total_sply_act_30d_cm',
 'macro_total_sply_act_5yr_cm',
 'macro_total_sply_act_7d_cm',
 'macro_total_sply_act_ever_cm',
 'macro_total_sply_adr_bal_usd_100_cm',
 'macro_total_sply_adr_bal_usd_10k_cm',
 'macro_total_sply_adr_bal_usd_1_cm',
 'macro_total_sply_adr_bal_usd_1m_cm',
 'macro_total_sply_adr_top_100_cm',
 'macro_total_sply_adr_top_1pct_cm',
 'macro_total_sply_ff_cm',
 'macro_total_tx_tfr_cnt_cm',
 'macro_total_tx_tfr_val_adj_usd_cm',
 'macro_total_tx_tfr_val_usd_cm',

 'macro_us_ex_open_interest_future_usd_cm',
 'macro_us_ex_volume_future_usd_cm',
 'macro_us_ex_volume_spot_usd_cm',


In [9]:
# TODO CLEAN MACRO CMC COLUMNS

# -confirm column ranges
# -confirm missingness
# -would it be better to have the raw missing from one of the data sources that i am using?
# -would it be helpful to be able to cross check a small number of assets for critical varaibles
#  like supply, mcap, addresses, transfers then if so, pull in raw CM panel
# -consider combining with others?
# -consider colsolidating down? or mayb wait until feat eng time?

[


 'macro_active_cryptos_cmc',
 'macro_active_exchanges_cmc',
 'macro_active_market_pairs_cmc',
 'macro_altcoin_usd_volume_24h_cmc',
 'macro_btc_dominance_cmc',
 'macro_ex_num_pairs_cex_cmc',
 'macro_ex_num_pairs_dex_cmc',
 'macro_ex_usd_volume_24h_cex_cmc',
 'macro_ex_usd_volume_24h_dex_cmc',
 'macro_total_usd_mcap_cmc',
 'macro_total_usd_volume_24h_cmc',

 ]

In [ ]:
# TODO CLEAN MACRO SAN COLUMNS

# -confirm column ranges
# -confirm missingness
# -would it be better to have the raw missing from one of the data sources that i am using?
# -would it be helpful to be able to cross check a small number of assets for critical varaibles
#  like supply, mcap, addresses, transfers then if so, pull in raw CM panel
# -consider combining with others?
# -consider colsolidating down? or mayb wait until feat eng time?


 'macro_aave_med_borrow_apy_san',
 'macro_aave_med_supply_apy_san',
 'macro_aave_med_variable_borrow_apy_san',
 'macro_btc_mvrv_san',
 'macro_eth_avg_fee_san',
 'macro_eth_median_fee_san',
 'macro_eth_mvrv_san',
 'macro_eth_roi_san',
 'macro_eth_stakers_count_san',
 'macro_eth_total_fee_san',
 'macro_funding_rate_med_usdt_binance_san',
 'macro_mcd_avg_liq_san',
 'macro_mcd_med_collat_ratio_san',
 'macro_mvrv_med_san',
 'macro_total_aave_borrowed_san',
 'macro_total_aave_deposits_san',
 'macro_total_aave_liq_san',
 'macro_total_aave_new_debt_san',
 'macro_total_aave_supply_san',
 'macro_total_compound_borrowed_san',
 'macro_total_compound_deposits_san',
 'macro_total_compound_liq_san',
 'macro_total_compound_new_debt_san',
 'macro_total_compound_supply_san',
 'macro_total_dai_created_san',
 'macro_total_dai_repaid_san',
 'macro_total_dex_volume_san',
 'macro_total_maker_borrowed_san',
 'macro_total_maker_deposits_san',
 'macro_total_maker_supply_san',
 'macro_total_nft_retail_trades_san',
 'macro_total_nft_retail_volume_san',
 'macro_total_nft_trades_san',
 'macro_total_nft_volume_san',
 'macro_total_nft_whale_trades_san',
 'macro_total_nft_whale_volume_san',
 'macro_total_open_interest_usdt_binance_san',
 'macro_total_open_value_usdt_binance_san',
 'macro_total_uni_claims_san',

In [ ]:
# TODO ENSURE NO MISSING, OTHERWISE FFILL OR SOMETHING

In [12]:
# TODO SORT ROWS AND COLUMNS


In [ ]:
# output panel

# TODO BUILD PANEL AT HOURLY LEVEL AS I CAN ADJUST LATER